## Leitura e Preprocessamento dos Dados

### Requirements

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Leitura e Concatenação 

In [2]:
path = os.getcwd()
files = glob.glob(os.path.join(path, "output/*/*.json"))

In [3]:
jsons = [pd.read_json(file, lines=True) for file in files]
df = pd.concat(jsons, ignore_index=True)

df["subject"] = [os.path.split(os.path.dirname(file))[-1] for file in files]

In [4]:
df

,title,abstract,subject
0,Modelagem de reforma catalítica seca de metano...,Estamos vivenciando o início da Era da Informa...,machinelearning
1,Aplicação de machine learning para apostas esp...,As práticas esportivas surgiram de atividades ...,machinelearning
2,Social machines: a unified paradigm to describ...,"The open, distributed approach of the Web and ...",machinelearning
3,Um método de aprendizagem seqüencial com filtr...,"Em aplicações de aprendizagem de máquina, é co...",machinelearning
4,Development of machine and deep learning based...,Artificial intelligence-based algorithms have ...,machinelearning
5,Automação de controle de acesso por reconhecim...,Esta pesquisa tem como objetivo propor um méto...,machinelearning
6,Visão computacional aplicada a gestão da manut...,O desgaste progressivo de peças rotativas como...,machinelearning
7,A machine learning-based methodology for autom...,Oil refineries process hazardous substances at...,machinelearning
8,Time Aware Sigmoid Optimization : a new learni...,The correct choice of hyperparameters for the ...,machinelearning
9,Time series forecasting with deep forest regre...,A time series is a collection of ordered obser...,machinelearning


In [5]:
df.describe()

,title,abstract,subject
count,20,20,20
unique,20,20,2
top,Modelagem de reforma catalítica seca de metano...,Estamos vivenciando o início da Era da Informa...,machinelearning
freq,1,1,10


### Stopwords

English and portuguese

In [6]:
languages = ['english', 'portuguese']

In [7]:
list_stop = stopwords.words(languages)
print(list_stop[:10])
print(list_stop[-10:])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
['tivessem', 'tivéssemos', 'tu', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês', 'vos']


In [9]:
documents = df["abstract"].copy()

documents_tokens = []
vocabulary = set()

for document in documents:
    document_lower = document.lower()  # transforma todos as frases em minusculo
   
    for language in languages: # para cada linguagem executa o list comprehension abaixo:
        token_list = [
            token for token in word_tokenize(document_lower, language=language) if token.isalnum() and token not in list_stop
        ] # quebra as frases em tokens, verifica se não está no stoplist, verifica se é alfanumerico. Caso sim, adiciona a lista de tokens
        
        if token_list not in documents_tokens:
            documents_tokens.append(token_list) #se a lista de tokens não constar na lista de frases, é adicionada
            
        for token in token_list:
            if token not in vocabulary and token.isalnum(): # se o token não já constar no vocabulario, adiciona no vocabulario
                vocabulary.add(token)

In [10]:
len(vocabulary) #Tamanho do vocabulario 

1565

In [11]:
len(documents_tokens) #Tamanho da lista de frases

20

In [18]:
print([v for v in vocabulary if len(v) == 1]) #Tokens indesejados

['b', 'k', '1', '3', 'á', '2', '5', '7', 'f']
